In [12]:
import os
import time
import gymnasium
import numpy as np
import optuna
import torch
from assignment3.Section1.MountainCarContinuous.actor_critic_car_loop import actor_critic

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [30]:
def objective(trial):
    # Define hyperparameter space
    env_name = "MountainCarContinuous-v0"
    log_dir = "runs/actor_critic"

    hidden_sizes = trial.suggest_categorical('hidden_sizes', [
        (64,), (128,), (256,), (128, 64), (256, 128)
    ])
    alpha = trial.suggest_float('alpha', 1e-5, 1e-2, log=True)
    episodes = 1000
    gamma = trial.suggest_float('gamma', 0.8, 0.99)
    entropy_coeff = trial.suggest_float('entropy_coeff', 1e-5, 1e-1, log=True)
    start_noise_std = trial.suggest_float('start_noise_std', 0.01, 0.3)
    end_noise_std = trial.suggest_float('end_noise_std', 0.01, 0.3)
    noise_decay = trial.suggest_float('noise_decay', 0.99, 0.9999)

    # Log the hyperparameters being used in this trial
    print(f"Trial {trial.number}:")
    print(f"  hidden_sizes: {hidden_sizes}")
    print(f"  alpha: {alpha}")
    print(f"  gamma: {gamma}")
    print(f"  entropy_coeff: {entropy_coeff}")
    print(f"  start_noise_std: {start_noise_std}")
    print(f"  end_noise_std: {end_noise_std}")
    print(f"  noise_decay: {noise_decay}")

    # Train the agent with these hyperparameters
    policy_net, value_net, rewards, train_time, avg_reward_50 = actor_critic(
        env_name=env_name,
        input_dim=6,  # MountainCarContinuous state dimension
        output_dim=3,  # Single action dimension
        hidden_sizes_theta=hidden_sizes,
        hidden_sizes_w=hidden_sizes,
        alpha_theta=alpha,
        alpha_w=alpha,
        episodes=episodes,
        gamma=gamma,
        entropy_coeff=entropy_coeff,  # Entropy bonus coefficient
        start_noise_std=start_noise_std,  # Initial noise
        end_noise_std=end_noise_std,  # Final noise
        noise_decay=noise_decay,  # Decay rate for noise
        log_dir=f"{log_dir}/{env_name}_g{gamma}_at{alpha}"
    )

    # Log the result of this trial
    print(f"  Average Reward: {avg_reward_50}")
    print(f"  Training Time: {train_time:.2f} seconds\n")

    return avg_reward_50

In [31]:
# Create a study object and specify the direction of optimization
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())

# Optimize the objective function over a number of trials
study.optimize(objective, n_trials=1000, timeout=60 * 60 * 20)  # e.g., 100 trials or 1 hour

# Print the best hyperparameters
print("Best Hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")
print(f"Best Average Reward: {study.best_value}")

[I 2025-01-02 21:43:18,867] A new study created in memory with name: no-name-64039150-2dc9-4912-b259-018a3a8915cb


Trial 0:
  hidden_sizes: (128,)
  alpha: 0.0024352069422591835
  gamma: 0.9532889056602523
  entropy_coeff: 3.558978004690944e-05
  start_noise_std: 0.1571416650213371
  end_noise_std: 0.058061693879282984
  noise_decay: 0.9983021693682942
Episode 1: Reward=-48.78, Avg(100)=-48.78, Avg(50)=-48.78, Noise STD=0.1571
Episode 2: Reward=-91.30, Avg(100)=-70.04, Avg(50)=-70.04, Noise STD=0.1569
Episode 3: Reward=-92.05, Avg(100)=-77.38, Avg(50)=-77.38, Noise STD=0.1566
Episode 4: Reward=-92.68, Avg(100)=-81.20, Avg(50)=-81.20, Noise STD=0.1563
Episode 5: Reward=-93.70, Avg(100)=-83.70, Avg(50)=-83.70, Noise STD=0.1561
Episode 6: Reward=23.33, Avg(100)=-65.86, Avg(50)=-65.86, Noise STD=0.1558
Episode 7: Reward=-91.04, Avg(100)=-69.46, Avg(50)=-69.46, Noise STD=0.1555
Episode 8: Reward=11.03, Avg(100)=-59.40, Avg(50)=-59.40, Noise STD=0.1553
Episode 9: Reward=-92.14, Avg(100)=-63.04, Avg(50)=-63.04, Noise STD=0.1550
Episode 10: Reward=-93.62, Avg(100)=-66.09, Avg(50)=-66.09, Noise STD=0.1548
E

[I 2025-01-02 22:38:00,770] Trial 0 finished with value: -inf and parameters: {'hidden_sizes': (128,), 'alpha': 0.0024352069422591835, 'gamma': 0.9532889056602523, 'entropy_coeff': 3.558978004690944e-05, 'start_noise_std': 0.1571416650213371, 'end_noise_std': 0.058061693879282984, 'noise_decay': 0.9983021693682942}. Best is trial 0 with value: -inf.


Episode 1000: Reward=-99.90, Avg(100)=-99.90, Avg(50)=-99.90, Noise STD=0.0581
  Average Reward: -inf
  Training Time: 3281.89 seconds

Trial 1:
  hidden_sizes: (128, 64)
  alpha: 1.1944975561737275e-05
  gamma: 0.8021239727993724
  entropy_coeff: 1.3591923129287288e-05
  start_noise_std: 0.129751623354225
  end_noise_std: 0.2336624923587489
  noise_decay: 0.9957693003665727
Episode 1: Reward=-33.61, Avg(100)=-33.61, Avg(50)=-33.61, Noise STD=0.2337
Episode 2: Reward=-33.61, Avg(100)=-33.61, Avg(50)=-33.61, Noise STD=0.2337
Episode 3: Reward=-33.89, Avg(100)=-33.70, Avg(50)=-33.70, Noise STD=0.2337
Episode 4: Reward=-33.50, Avg(100)=-33.65, Avg(50)=-33.65, Noise STD=0.2337
Episode 5: Reward=-36.15, Avg(100)=-34.15, Avg(50)=-34.15, Noise STD=0.2337
Episode 6: Reward=-32.36, Avg(100)=-33.85, Avg(50)=-33.85, Noise STD=0.2337
Episode 7: Reward=-33.36, Avg(100)=-33.78, Avg(50)=-33.78, Noise STD=0.2337
Episode 8: Reward=-34.15, Avg(100)=-33.83, Avg(50)=-33.83, Noise STD=0.2337
Episode 9: Rew

[I 2025-01-02 23:33:13,625] Trial 1 finished with value: -inf and parameters: {'hidden_sizes': (128, 64), 'alpha': 1.1944975561737275e-05, 'gamma': 0.8021239727993724, 'entropy_coeff': 1.3591923129287288e-05, 'start_noise_std': 0.129751623354225, 'end_noise_std': 0.2336624923587489, 'noise_decay': 0.9957693003665727}. Best is trial 0 with value: -inf.


...

Episode 1000: Reward=59.60, Avg(100)=-84.37, Avg(50)=-86.24, Noise STD=0.2989
  Average Reward: -inf
  Training Time: 2931.60 seconds

Trial 23:
  hidden_sizes: (64,)
  alpha: 0.0036847767914245048
  gamma: 0.9714347497598761
  entropy_coeff: 5.610918332393168e-05
  start_noise_std: 0.1126742870573873
  end_noise_std: 0.16795132309230068
  noise_decay: 0.9978317466924482
Episode 1: Reward=-60.49, Avg(100)=-60.49, Avg(50)=-60.49, Noise STD=0.1680
Episode 2: Reward=35.58, Avg(100)=-12.46, Avg(50)=-12.46, Noise STD=0.1680
Episode 3: Reward=-94.48, Avg(100)=-39.80, Avg(50)=-39.80, Noise STD=0.1680
Episode 4: Reward=-92.84, Avg(100)=-53.06, Avg(50)=-53.06, Noise STD=0.1680
Episode 5: Reward=-93.21, Avg(100)=-61.09, Avg(50)=-61.09, Noise STD=0.1680
Episode 6: Reward=23.36, Avg(100)=-47.01, Avg(50)=-47.01, Noise STD=0.1680
Episode 7: Reward=-94.35, Avg(100)=-53.78, Avg(50)=-53.78, Noise STD=0.1680
Episode 8: Reward=55.22, Avg(100)=-40.15, Avg(50)=-40.15, Noise STD=0.1680
Episode 9: Reward=-94

[I 2025-01-03 17:20:47,570] Trial 23 finished with value: -inf and parameters: {'hidden_sizes': (64,), 'alpha': 0.0036847767914245048, 'gamma': 0.9714347497598761, 'entropy_coeff': 5.610918332393168e-05, 'start_noise_std': 0.1126742870573873, 'end_noise_std': 0.16795132309230068, 'noise_decay': 0.9978317466924482}. Best is trial 9 with value: 12.037644487601224.


Episode 1000: Reward=-99.90, Avg(100)=-99.90, Avg(50)=-99.90, Noise STD=0.1680
  Average Reward: -inf
  Training Time: 2730.12 seconds

Trial 24:
  hidden_sizes: (128, 64)
  alpha: 0.00013871319874410632
  gamma: 0.9191319288558385
  entropy_coeff: 0.0002692435286833037
  start_noise_std: 0.18077712135304377
  end_noise_std: 0.1316735914314744
  noise_decay: 0.9966354684861523
Episode 1: Reward=-49.39, Avg(100)=-49.39, Avg(50)=-49.39, Noise STD=0.1808
Episode 2: Reward=62.90, Avg(100)=6.75, Avg(50)=6.75, Noise STD=0.1802
Episode 3: Reward=-91.04, Avg(100)=-25.84, Avg(50)=-25.84, Noise STD=0.1796
Episode 4: Reward=-92.51, Avg(100)=-42.51, Avg(50)=-42.51, Noise STD=0.1790
Episode 5: Reward=-92.45, Avg(100)=-52.50, Avg(50)=-52.50, Noise STD=0.1784
Episode 6: Reward=-93.46, Avg(100)=-59.33, Avg(50)=-59.33, Noise STD=0.1778
Episode 7: Reward=-93.06, Avg(100)=-64.14, Avg(50)=-64.14, Noise STD=0.1772
Episode 8: Reward=-93.00, Avg(100)=-67.75, Avg(50)=-67.75, Noise STD=0.1766
Episode 9: Reward

[I 2025-01-03 18:11:34,420] Trial 24 finished with value: 6.754542688874871 and parameters: {'hidden_sizes': (128, 64), 'alpha': 0.00013871319874410632, 'gamma': 0.9191319288558385, 'entropy_coeff': 0.0002692435286833037, 'start_noise_std': 0.18077712135304377, 'end_noise_std': 0.1316735914314744, 'noise_decay': 0.9966354684861523}. Best is trial 9 with value: 12.037644487601224.


Episode 1000: Reward=-92.54, Avg(100)=-58.80, Avg(50)=-62.28, Noise STD=0.1317
  Average Reward: 6.754542688874871
  Training Time: 3046.84 seconds

Best Hyperparameters:
  hidden_sizes: (256,)
  alpha: 0.000432860862913505
  gamma: 0.9571389501004702
  entropy_coeff: 0.0075942800574955
  start_noise_std: 0.12655319079849925
  end_noise_std: 0.2507795275354682
  noise_decay: 0.9952850823459259
Best Average Reward: 12.037644487601224
